In [1]:
import pandas as pd
import numpy as np

In [ ]:
## taking the y variable with columns: country and unadjusted pay gap
y =  pd.read_csv("lonbg.csv")

In [5]:
y

,country,unadjusted_gender_pay_gap
0,Latvia,19.0
1,Austria,18.3
2,Czechia,18.0
3,Hungary,17.8
4,Germany,17.6
5,Estonia,16.9
6,Finland,16.8
7,Slovakia,15.7
8,Denmark,14.0
9,Greece,13.6


## Education

In [ ]:
## reading the data for the x variable education
education = pd.read_csv("tertiary_education.csv")

In [ ]:
## filtering the data for the year 2023
education["Tertiary_Education(%)"] = education["2023"]
education = education.drop(["2023"], axis=1)

In [ ]:
## Checking the countries which are not in the y variable
education[~education["country"].isin(y["country"])].dropna()

,country,Tertiary_Education(%)
0,EU,84.1
31,Serbia,93.7
32,Bosnia and Herzegovina,91.9
33,Turkey,76.5


In [ ]:
## adding the education data and the y variable in a new dataframe
df = pd.merge(y, education, how="left", left_on="country", right_on="country")

## GDP per capita PPS

In [ ]:
## reading the data for the x variable GDP
gdp = pd.read_csv("GDP per capita in PPS_tec00114.csv")

In [ ]:
## selecting the columns of interest and filtering for the year 2023
col=["Geopolitical entity (reporting)", "TIME_PERIOD", "OBS_VALUE"]
gdp = gdp[col][gdp["TIME_PERIOD"] == 2023]

## renaming the columns
gdp["GDP_perCapitaPPS"] = gdp["OBS_VALUE"]
gdp = gdp.drop(["OBS_VALUE", "TIME_PERIOD"], axis=1)

In [ ]:
## adding gdp data to the dataframe with education and y variable
df = pd.merge(df, gdp, how="left", left_on="country", right_on="Geopolitical entity (reporting)").drop(columns="Geopolitical entity (reporting)", axis=1)

## Region

In [ ]:
text = "Northern Europe: Denmark, Isle of Man, Estonia, Faroe Islands (Denmark), Finland (incl. Åland Islands), United Kingdom, Guernsey, Ireland, Jersey, Latvia, Lithuania, Norway, Svalbard and Jan Mayen, Iceland, Sweden./n Eastern Europe: Bulgaria, Hungary, Moldova, Ukraine, Poland, Romania, Russia (West of the Urals), Slovakia, Czech Republic, Belarus./n Southern Europe: Albania, Andorra, Bosnia-Herzegovina, Gibraltar, Greece, Italy, Kosovo, Croatia, Macedonia, Malta, Montenegro, Portugal, San Marino, Serbia, Slovenia, Spain, Turkey (European part), Vatican City./n Western Europe: Belgium, Germany, France, Liechtenstein, Luxembourg, Monaco, Netherlands, Austria, Switzerland."

In [ ]:
## reading the data for the x variable geographical regions
regions = pd.read_csv('world_regions.csv')

In [ ]:
## choosing the columns of interest and renaming them
col = ['Region Name', 'Sub-region Name', 'Country or Area']
regions = regions[col].drop(["Region Name"], axis=1).rename(columns={"Sub-region Name": "Geographical_region", "Country or Area": "country"})


# df = pd.merge(df, regions, how="left", left_on="country", right_on="country").drop(columns="country", axis=1)
# regions[col][regions["Country or Area"].isin(df["country"])]
# regions[col][regions['Region Name'] == 'Europe']

In [ ]:
## adding the geographical regions data to the dataframe
df = pd.merge(df, regions, how="left", left_on="country", right_on="country")

## Maternity leave

In [ ]:
## reading the data for the x variable maternity leave entitlements
maternity = pd.read_csv('PF2_5_Trends_in_leave_entitlements_around_childbirth__editting.xlsx')

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 14-15: invalid continuation byte